In [1]:
import numpy as np
import pandas as pd
import os
import time
import sys
import random

from pandas.tseries.offsets import Hour


#from sklearn.externals.joblib.memory import Memory, MemorizedFunc
from tempfile import mkdtemp
from datetime import datetime, timedelta
from colorama import Fore   # печатает в консоль цветом


from catboost import CatBoostRegressor, CatBoost, Pool
import pickle

from statsmodels import version
print('numpy = ',np.__version__)
print('pandas = ', pd.__version__)
print(sys.version)

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', family='Verdana')

%matplotlib inline

numpy =  1.12.1
pandas =  0.20.3
3.5.4 | packaged by conda-forge | (default, Aug 10 2017, 01:38:47) [MSC v.1900 64 bit (AMD64)]


In [2]:
class scale_class:
    def __init__(self):
        self.Xmin = 0
        self.Xmax = 0

    def scale(self, X):
        self.Xmin = X.min(axis=0)
        self.Xmax = X.max(axis=0)
        return (X - self.Xmin)/(self.Xmax-self.Xmin)

    def inverse(self, X_scaled):
        if X_scaled.ndim == 1:
            return X_scaled*(self.Xmax-self.Xmin)+self.Xmin
        else:
            d=X_scaled.shape[1]
            return X_scaled*(self.Xmax[:d]-self.Xmin[:d])+self.Xmin[:d]

In [3]:
# Класс для нормализации рядов
class normalize_class:

    # Преобразование - normalize
    def normalize(self, data):
        self.mean = data.mean(axis=0).values
        self.std = data.std(axis=0).values
        return (data - self.mean)/self.std

    # Обратное преобразование 
    def inverse(self, normalized_data):
        if normalized_data.ndim == 1:
            return normalized_data*self.std+self.mean
        else:
            d=normalized_data.shape[1]
            n = normalized_data*self.std[:d]+self.mean[:d]
            return n        

In [4]:
class printime_class:
    def __init__(self, flag=True):
        #from datetime import timedelta
        #from time import time
        self.flag = flag
        self.loop_number = 0
        self.t = [ time.time() ]

    def next(self, message=''):
        if self.flag:
            self.t.append( time.time() )
            print ('{0} {1}'.format(message, timedelta(seconds=round( self.t[-1]-self.t[-2], 0)) ))
    
    def overall(self, message=''):
        if self.flag:
            self.loop_number+=1
            self.t.append( time.time() )
            print ('{0} {1} {2}'.format(self.loop_number, timedelta(seconds=round( self.t[-1]-self.t[0], 0)), message ))

In [5]:
# Функция сохранения в файл модели - чтобы каждый раз не перерасчитывать
def save_model(fname, *model):
    directory = os.path.dirname(fname)
    if not os.path.isdir(directory): 
        os.makedirs(directory)
    if os.path.isfile(fname): 
        os.remove(fname)
    
    with open(fname,"wb") as fout:
        pickle.dump(model, fout)

### Тело программы

In [6]:
NOTE=printime_class(True)

In [7]:
ydat = pd.read_csv('../data/aggregated_hours_3week_2009.01_2016.06.csv', parse_dates=True, index_col='date', dtype='int32')

ynormer = normalize_class()

ydat_norm = ynormer.normalize(ydat) 

In [8]:
# Делим на обучающую и тестовую выборки

# С января 2009 по апрель 2016 будем обучать регрессию
#train = ydat['2016, 4, 30 ':'2016, 4, 30 , 17 '].index
#train = ydat['2016':'2016, 4, 30 , 17 '].index
train = ydat['2011':'2016, 4, 30 , 17 '].index

# Апрель 2016 
april = ydat['2016, 4'].index
april_test_per = ydat['2016, 3, 31, 23': '2016, 4, 30 , 17 '].index

mar = ydat['2016, 3'].index
# Май - 
may = ydat['2016, 5'].index
may_test_per = ydat['2016, 4, 30, 23': '2016, 5, 31 , 17 '].index

# Июнь
june = ydat['2016, 6'].index
june_test_per = ydat['2016, 5, 31, 23': '2016, 6, 30 , 17 '].index

# 2016 год - для разных просмотров
year2016 = ydat['2016, 1, 1': '2016, 6, 30 , 23 '].index

In [9]:
# Делаем отступ от начала данных
y=ydat.loc[train[0]:]
y_norm = ydat_norm[train[0]:]

In [10]:
%%time


# генерация признаков
# Будем строить 102 модели, в которых шаг (горизонт прогнозирования 1-6 часов) - это категориальный признак

# Я пробовал строить как в задании, 6 моделей, где регион - это категориальный признак, 
# но результат слабый, видимо из-за того, что ряды регионов разной формы
# и в памяти не помещается, поэтому очень долго считалось

steps = np.arange(1,7)
nsteps=len(steps)

Wall time: 0 ns


In [11]:
%%time
# Загрузка данных из 4 недели
week4_april = pd.read_csv('../4week/week4_answer_april.csv')
week4_may = pd.read_csv('../4week/week4_answer_may.csv')
week4_june = pd.read_csv('../4week/week4_answer.csv')

temp_week4 = pd.concat( [week4_april, week4_may, week4_june], copy=False )


temp = pd.DataFrame.from_records(list(np.char.split(list(temp_week4.id), '_')), 
                                 columns=['reg', 'date', 'hour', 'step'])

temp.index = pd.to_datetime(temp.date.values+' '+temp.hour.values+':00')
temp.step = temp.step.astype('int32')
temp['week4'] = temp_week4.y.values


week4 = []

for step in steps:
    step_temp = pd.DataFrame()
    for reg in y:
        step_temp[reg] = temp.loc[(temp.reg==reg) & (temp.step==step), 'week4']
    week4.append(step_temp.reindex_like(other=y).fillna(0))

del(temp, temp_week4, step_temp)

Wall time: 1min 40s


In [12]:
# признаки, одинаковые для всех регионов и steps (горизонтов)
def init_base_features():
    base_features = pd.DataFrame(index=y.index)

    trend = pd.Series(data=np.arange(1, len(base_features)+1), index=base_features.index)
    
    K=5

    # Недельные синусоиды
    for i in np.arange(1, K+1):
        base_features['week_sin'+str(i)] = np.sin( trend*(2*np.pi*i/168.))
        base_features['week_cos'+str(i)] = np.cos( trend*(2*np.pi*i/168.))

    # Годовые синусоиды
    for i in np.arange(1, 3*K+1):
        base_features['year_sin'+str(i)] = np.sin( trend*(2*np.pi*i/8766.))
        base_features['year_cos'+str(i)] = np.cos( trend*(2*np.pi*i/8766.))

    # Тренд
    base_features['trend'] = trend

    # *******************************************
    # Константа
    base_features['const'] = np.ones(len(base_features)) 
        
    # Категориальные признаки

   # Признаки дня недели
    base_features['cat_dow'] = base_features.index.dayofweek
    base_features = base_features.join( pd.get_dummies(base_features.cat_dow, prefix='dow'))

    #Признаки номера недели в году
    base_features['cat_nweek'] = base_features.index.weekofyear
    base_features = base_features.join( pd.get_dummies(base_features.cat_nweek, prefix='nweek'))

    # признаки месяца
    base_features['cat_month'] = base_features.index.month
    base_features = base_features.join( pd.get_dummies(base_features.cat_month, prefix='month'))

    # Признак часа
    base_features['cat_hour'] = base_features.index.hour
    base_features = base_features.join( pd.get_dummies(base_features.cat_hour, prefix='hour'))

    # Признак начало месяца
    base_features['cat_is_month_start'] = base_features.index.is_month_start
    base_features = base_features.join( pd.get_dummies(base_features.cat_is_month_start, prefix='month_start'))

    # Признак конца года
    base_features['cat_is_year_end'] = base_features.index.is_year_end
    base_features = base_features.join( pd.get_dummies(base_features.cat_is_year_end, prefix='year_end'))

    # Признак начала года
    base_features['cat_is_year_start'] = base_features.index.is_year_start
    base_features = base_features.join( pd.get_dummies(base_features.cat_is_year_start, prefix='year_start'))


    #*************************************************************************
    # Признак Аномалия - январь 2016

    dates = y['2016-01-23 06:00':'2016-01-24 7:00'].index
    base_features['cat_anomaly2016'] = [1 if x in dates else 0 for x in base_features.index ]
    base_features = base_features.join( pd.get_dummies(base_features.cat_anomaly2016, prefix='anomaly'))
    del(dates)

    # Конец объявления  признаков *****************************
        
    return base_features


In [13]:
# признаки, которые зависят от региона, не зависят от шага
def init_reg_features_list():
    features_list= []

    reg_period= y.index
    # ненормированные
    get_previous=pd.DataFrame(data=np.zeros(y.shape), index=y.index, columns=y.columns)
    for i in range(1,23):
        get_previous += ydat.loc[reg_period.shift(-i, freq='h')].values
        features_list.append( get_previous )

    del(get_previous)

    def get_sum(num):
        result = pd.DataFrame(data=np.zeros(y.shape), index=y.index, columns=y.columns)
        result_norm = pd.DataFrame(data=np.zeros(y.shape), index=y.index, columns=y.columns)
        
        for i in range(1,num+1):
            result += ydat.loc[reg_period.shift(-i, freq='h')].values
            result_norm += ydat_norm.loc[reg_period.shift(-i, freq='h')].values
        features_list.append( result )
        features_list.append( result_norm )
        del(result, result_norm)


    # сутки, неделю, месяц
    #ненормированные
    get_sum(24)
    get_sum(24*7)
    get_sum(24*30)
    
    for i in np.arange(1,25):
        # Количество поездок в моменты времени 1-24 .... часов назад
        features_list.append( pd.DataFrame(data=ydat.loc[reg_period.shift(-i, freq='h')].values, 
                                           index=y.index, columns=y.columns))
        features_list.append( pd.DataFrame(data=ydat_norm.loc[reg_period.shift(-i, freq='h')].values, 
                                           index=y.index, columns=y.columns))

    
    
    
    
    return features_list
   

In [14]:
# Признаки, которые зависят от шага для всех регионов
def init_step_features_dict():
    features_dict = {}

    for step__ in np.arange(1,7):

        features_dict[str(step__)] = []
        # Количество поездок в моменты времени 24, 48.... часов назад
        for Kd in np.arange(24, 24*4, 24):
            features_dict[str(step__)].append(pd.DataFrame(data=ydat.loc[y.index.shift(-Kd+step__, freq='h')].values,
                                                           index=y.index, columns=y.columns ))
        # Результат 4 недели 
        features_dict[str(step__)].append(week4[step__-1])

    return features_dict
    
    

In [15]:
%%time
# Здесь формируются все признаки, они постоянно находятся в памяти, для скорости
base_features = init_base_features()
reg_features_list = init_reg_features_list()
step_features_dict = init_step_features_dict()

Wall time: 1min 27s


In [16]:
# get_features на каждом шаге для каждого региона формирует матрицу признаков для расчета отдельного региона

def get_features(reg, step):
    features = base_features
    
    for i, frame in enumerate(reg_features_list):
        features['reg{}'.format(i)] = frame[reg]
    
    for frame in step_features_dict[str(step)]:
        features['step{}'.format(i)] = frame[reg]
    return features
    

In [18]:
%%time

week5 = pd.DataFrame()

bad_regs_list = []
Q_list = []

i=0
for step in steps:

    # Метка
    ylabel = pd.DataFrame(data=ydat.loc[y.index.shift(step, freq='h')].values, index=y.index, columns=y.columns)
    
    week5_reg = pd.DataFrame(index=june_test_per)

    for reg in y.columns:
        i+=1
        NOTE.next('start loop.............'+str(i))

        # Формируем признаки
        reg_features = get_features(reg, step)
        # Список категориальных признаков и их названий cat_features, cat_features_names
        if i == 1:
            cat_features = np.ravel( np.where( np.char.startswith(list(reg_features.columns), prefix='cat_') ) )
            cat_features_names = reg_features.columns.values[ cat_features]


        model = CatBoostRegressor(loss_function='RMSE',iterations=10000, depth=8, learning_rate=0.03,
                             od_type='Iter', od_wait=40, has_time=True, # od_pval=10e-7,
                             task_type='CPU', thread_count=4, rsm=0.1,
                             l2_leaf_reg=3,
                             random_seed=1000,
                             train_dir='./catboost_train_dir')



        train_pool = Pool(data= reg_features.loc[train], label = ylabel.loc[train, reg], 
                          has_header=True, cat_features=cat_features,  thread_count=4)

        eval_pool = Pool(data= reg_features.loc[may_test_per], label = ylabel.loc[may_test_per, reg], 
                         has_header=True, cat_features=cat_features,  thread_count=4)    

        june_pool = Pool(data= reg_features.loc[june_test_per], has_header=True, 
                         cat_features=cat_features,  thread_count=4)    


        model.fit(X=train_pool, use_best_model=True, eval_set=eval_pool, 
                  logging_level='Silent')

        week5_reg[reg] = np.round(model.predict(june_pool), 0).astype('int')

        week5_reg[reg].loc[week5_reg[reg] < 0] = 0

        Q_table = abs(week5_reg - ylabel.loc[week5_reg.index, week5_reg.columns]).dropna(axis=0) 

        Q = np.sum(Q_table.values) / np.product(Q_table.shape)
        Q_reg = np.sum(Q_table[reg].values) / (len(Q_table))

        #plt.scatter(x=Q_reg, y=mean_shift, label='y_')


        if Q_reg > 15:
            bad_regs_list.append([reg, Q_reg])

        print('Q_reg= {}, reg= {}'.format(Q_reg, reg))

        NOTE.next('iterations {} '.format(model.tree_count_))
        NOTE.overall(' loop, Q= {} \n\n'.format(Q) )

        del(reg_features, model, train_pool, eval_pool, june_pool, Q_table)

    Q_list.append( Q )
    print('gained Q= {} for step {}'.format( sum(Q_list)/len(Q_list), step ))
    week5_reg.insert(loc=0, column='step', value=np.full(len(week5_reg), fill_value=step))
    week5=week5.append(week5_reg)

    

start loop.............1 0:01:33
Q_reg= 9.955244755244756, reg= 1075
iterations 1735  0:05:49
5 0:34:44  loop, Q= 9.955244755244756 


start loop.............2 0:00:00
Q_reg= 14.20979020979021, reg= 1076
iterations 722  0:02:25
6 0:37:09  loop, Q= 12.082517482517483 


start loop.............3 0:00:00
Q_reg= 10.923076923076923, reg= 1077
iterations 1192  0:03:40
7 0:40:49  loop, Q= 11.696037296037296 


start loop.............4 0:00:00
Q_reg= 8.637762237762237, reg= 1125
iterations 1600  0:05:08
8 0:45:58  loop, Q= 10.931468531468532 


start loop.............5 0:00:00
Q_reg= 19.053146853146853, reg= 1126
iterations 843  0:02:43
9 0:48:41  loop, Q= 12.555804195804196 


start loop.............6 0:00:00
Q_reg= 22.762237762237763, reg= 1127
iterations 2183  0:06:55
10 0:55:37  loop, Q= 14.256876456876457 


start loop.............7 0:00:00
Q_reg= 27.186013986013986, reg= 1128
iterations 908  0:02:37
11 0:58:14  loop, Q= 16.103896103896105 


start loop.............8 0:00:00
Q_reg= 30.482

In [20]:
week5.head()

,step,1075,1076,1077,1125,1126,1127,1128,1129,1130,...,1630,1684,1733,1734,1783,2068,2069,2118,2119,2168
date,,,,,,,,,,,,,,,,,,,,,
2016-05-31 23:00:00,1,32,43,30,32,86,176,238,268,344,...,6,2,3,194,79,94,11,201,37,90
2016-06-01 00:00:00,1,17,23,9,24,46,148,172,203,356,...,8,1,1,57,10,30,5,107,23,40
2016-06-01 01:00:00,1,8,10,5,13,23,70,92,122,268,...,7,1,1,5,1,12,1,25,16,11
2016-06-01 02:00:00,1,5,7,1,9,19,49,65,91,230,...,6,0,1,2,3,5,0,30,3,2
2016-06-01 03:00:00,1,4,13,5,10,26,46,54,81,148,...,7,1,1,3,2,6,0,27,1,4


In [21]:
#week5 = ynormer.inverse(week5)

week5_answer = pd.DataFrame()

frame = pd.DataFrame(index = week5.index)
for reg in y:
    frame['y'] = week5[reg]
    frame['step']=week5.step
    frame['reg'] = np.full(len(week5), fill_value=reg)
    week5_answer=week5_answer.append(frame)

#week5_answer = yscale.inverse(week5_answer)

idx_date = week5_answer.index
idx_step = week5_answer['step']

id_ = week5_answer.reg +'_' \
    + idx_date.date.astype('str')+'_' \
    + idx_date.hour.astype('str')+'_' \
    + idx_step.astype('str')

week5_answer.insert(loc=0, column='id', value=id_)
week5_answer['date'] = idx_date
week5_answer['step'] = idx_step

week5_answer.sort_values(['reg', 'date', 'step'], inplace=True)

#week5_answer_copy = week5_answer.copy()

week5_answer.drop(['date', 'step', 'reg'], axis=1, inplace=True)

week5_answer.to_csv('./week5_answer.csv', index=False)



In [22]:
week4_june = pd.read_csv('../4week/week4_answer.csv')
np.all(week4_june.id.values == week5_answer.id.values)


True

In [ ]:
#***************************************************

# Ссылка на сабмишн 

https://inclass.kaggle.com/c/yellowtaxi

Сергей Новожилов